# Predicting Position of NBA Players
### Based on in game stats

In [2]:
import pandas as pd

### Creating Dataframe

In [3]:
shot_df = pd.read_csv("./csvData/NBA_Shot_dist - Sheet2.csv")
guard_df = pd.read_csv("./csvData/NBA_Shot_dist - guard_list.csv")
forward_df = pd.read_csv("./csvData/NBA_Shot_dist - forwards_list.csv")
center_df = pd.read_csv("./csvData/NBA_Shot_dist - center_list.csv")
heightWeight_df = pd.read_csv("./csvData/playerHeight.csv")

heightWeight_df = heightWeight_df.iloc[:,[0,7,9]]

shot_df.columns = ['name', 'team', 'szn', 'type', 'games', 
              '0to8_make', '8to16_make', '16to24_make', '24plus_make', 'bcMake',
              '0to8_att', '8to16_att', '16to24_att', '24plus_att', 'bcatt', 
              'avg_dist', 'avg_madeDist', 'avg_missDist']

gen_df = pd.read_csv("./csvData/NBA_Shot_dist - gen_stats.csv")
gen_df = gen_df.iloc[:, [0,19,20,22,23]]
gen_df.columns = ['name','reb','ast','stl','blk',]

position_df = pd.concat([guard_df, forward_df, center_df])
position_df = position_df.groupby(['name'], as_index = False).sum()

In [4]:
df = pd.merge(heightWeight_df, shot_df, how='inner')
df = pd.merge(df,gen_df, how='inner')
df = pd.merge(df, position_df, how='inner')

### Deleting Columns

In [5]:
del df['0to8_att']
del df['8to16_att']
del df['16to24_att']
del df['24plus_att']
del df['bcatt']
del df['bcMake']
del df['avg_missDist']
del df['avg_dist']
del df['team']
del df['szn']
del df['type']

   ### Tuning and Normalizing
   * adding avg_madeDist^2 col

### Creating X and y - Train and Test Sets

In [6]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, 1:13]

y = df.iloc[:, 13:16]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)


### Random Forest Classifier

In [7]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn import metrics


""" RANDOM FOREST PREDICTION """

# X, y = make_classification(n_samples=100, n_features=5, n_informative=5, n_redundant=0, random_state=0, shuffle=False)

clf = RandomForestClassifier(n_estimators=120, max_depth=15, random_state=0)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


print(metrics.accuracy_score(y_test, y_pred), " -- Accuracy with Random Forests\n")

print("-------------- Importances per column with RF")

print((clf.feature_importances_), "\n")

0.7368421052631579  -- Accuracy with Random Forests

-------------- Importances per column with RF
[0.32199377 0.18964395 0.02764229 0.04623753 0.03016824 0.03600212
 0.04269972 0.09406097 0.0576071  0.064818   0.03183799 0.05728833] 



### Splitting Y's up
* y1 = guard
* y2 = forward
* y3 = center

In [9]:
y1 = y.iloc[:,0]
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=.3)
#y2 = y.iloc[:,1]
#X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=.3)
#y3 = y.iloc[:,2]
#X_train, X_test, y3_train, y3_test = train_test_split(X, y3, test_size=.3)




In [10]:
""" RANDOM FOREST PREDICTION """

# X, y = make_classification(n_samples=100, n_features=5, n_informative=5, n_redundant=0, random_state=0, shuffle=False)

clf = RandomForestClassifier(n_estimators=120, max_depth=15, random_state=0)

clf.fit(X_train, y1_train)

y1_pred = clf.predict(X_test)


print(metrics.accuracy_score(y1_test, y1_pred), " -- Accuracy with Random Forests\n")

print("-------------- Importances per column with RF")

print((clf.feature_importances_), "\n")

0.9210526315789473  -- Accuracy with Random Forests

-------------- Importances per column with RF
[0.37431235 0.22350299 0.01632842 0.03647583 0.017066   0.01972231
 0.0214974  0.04985657 0.09017561 0.0745348  0.02358435 0.05294335] 



### Using KMeans to determine how many groups to make
* Make 20 models
* Group by group number
* sum by position
* get score of model
* check 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# Use silhouette score to find optimal number of clusters to segment the data
num_clusters = np.arange(2,10)
results = {}
for size in num_clusters:
    model = KMeans(n_clusters = size).fit(X)
    predictions = model.predict(X)
    results[size] = silhouette_score(X, predictions)

best_size = max(results, key=results.get)